In [ ]:
import sys
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as lg
import matplotlib
import matplotlib.pyplot as plt
from cfd_common import *
from math import *

In [ ]:
%matplotlib qt

In [ ]:
def VelocityGhostPoints(u,v):
    ### left     
    u[0,  :] = -u[1,  :]    # on a envie de mettre 0, mais ne pas oublier que l'on est sur une grille alternée,
                            # donc la frontière passe entre i=-1 (ghost point) et i=0  =>  la *moyenne* de ces
                            # deux points doit être =0, pas le ghost point lui même
    u[0,  30:60] = 2 - u[1,  30:60]    # pareil, on a envie de mettre 1, mais c'est la moyenne qui compte

    u0 = np.zeros_like(u[0,:])
    u0[1:-1] = 1*np.exp(-((y-0.6*LY)/0.15)**8)
    u[0, :] = 2*u0 - u[1, :]

    v[0,  :] = -v[1,  :]    # "0" comme au dessus (pas de vitesse verticale)
    ### right    
    u[-1, :] = u[-2, :]    # outflow condition => dérivée nulle
    v[-1, :] = v[-2, :]    # outflow condition => dérivée nulle
    ### bottom
    u[:,  0] = 0  # no slip
    v[:,  0] = 0  # imperméabilité
    ### top     
    u[:, -1] = 0  # no slip
    v[:, -1] = 0  # imperméabilité

        
def PhiGhostPoints(phi):
    """
    copie les points fantomes
    tjrs Neumann

    global ==> pas de return 

    """
    ### left
    phi[0,  :] = phi[1,  :]
    ### right             
    phi[-1, :] = -phi[-2, :]
    ### bottom               
    phi[:,  0] = phi[:,  1]
    ### top            
    phi[:, -1] = phi[:,  -2]

In [ ]:
plt.ion()

In [ ]:


###### Domain Size

LY = 2.
LX = 4.

###### Grid Size

### (incuding ghost points)

NX = 400
NY = 125

# Number of iterations for the divergence
NI=1


### Taille du domaine reel
nx = NX-2
ny = NY-2

### Nombre d'iterations
nitermax = int(50001)

### Modulo
modulo = 50

###### CONDITIONS INITIALES

##### Valeurs initiales des vitesses
u = np.zeros((NX,NY)) 
v = np.zeros((NX,NY))

###### Elements differentiels 

Δx = LX/nx
Δy = LY/ny


### Grid, for plotting only
x = np.linspace(Δx/2,LX-Δx/2,nx) 
y = np.linspace(Δy/2,LY-Δy/2,ny)
[xx,yy] = np.meshgrid(x,y) 

### ATTENTION: dt_init calculer la CFL a chaque iteration... 
Δt = 0.002

t = 0. # total time

# parameters (Reynolds number)
Re = 80000


### Tableaux avec points fantomes
### Matrices dans lesquelles se trouvent les extrapolations
ADVu = np.zeros((NX,NY))
ADVv = np.zeros((NX,NY))

### Definition des matrices ustar et vstar
ustar = np.zeros((NX,NY))
vstar = np.zeros((NX,NY))

### Definition de divstar
divstar = np.zeros((NX,NY))

### Definition de la pression phi
phi      = np.zeros((NX,NY))
gradphix = np.zeros((NX,NY))
gradphiy = np.zeros((NX,NY))


###### CONSTRUCTION des matrices et LU decomposition

### Matrix construction for projection step
LAPN = FD_2D_Laplacian_matrix (nx, ny, Δx, Δy, BCdir_left=False, BCdir_right=True, BCdir_top=False, BCdir_bot=False) 
LUPN = LUdecomposition(LAPN)



################
###### MAIN LOOP 
tStart = t

plt.figure(figsize=(14,7))

for niter in range(nitermax):
        
    t += Δt

####    for k in range(NI):

    ###### Advection semi-Lagrangienne
    ADVu=SemiLag(u,v,u, Δx,Δy,Δt)
    ADVv=SemiLag(u,v,v, Δx,Δy,Δt)

    ###### Diffusion step
    ustar = ADVu + Δt*Laplacian(u, Δx,Δy)/Re
    vstar = ADVv + Δt*Laplacian(v, Δx,Δy)/Re
        
    ###### Ghost points update
    VelocityGhostPoints(ustar,vstar)
 
    ### Update divstar 
    divstar = Divergence(ustar,vstar, Δx,Δy)
#    divstar = divstar - np.mean(divstar[1:-1,1:-1])
    
    ### Solving the linear system
    phi[1:-1,1:-1] = Resolve(LUPN, RHS=divstar[1:-1,1:-1])

    ### update Pressure ghost points 
    PhiGhostPoints(phi)

    ### Update gradphi
    gradphix[1:-1, :] = (phi[2:, :] - phi[:-2, :])/Δx/2
    gradphiy[:, 1:-1] = (phi[:, 2:] - phi[:, :-2])/Δy/2

    ### Project u
    u = ustar - gradphix
    v = vstar - gradphiy

    ###### Mise a jour des points fantomes
    ###### pour le champ de vitesse et T

    VelocityGhostPoints(u,v)

    if ((niter+1)%modulo==0):
        
        plt.clf()
        plt.title(r"Velocity field for $Re={}$, $u_0=1.0$, at $t={:.1f}$".format(Re,t))
        
        # Get the colormap colors, multiply them with the factor "a", and create new colormap
        my_cmap = plt.cm.gnuplot2_r(np.arange(plt.cm.gnuplot2_r.N))
        my_cmap = 0.5 * (1+my_cmap)
        my_cmap = matplotlib.colors.ListedColormap(my_cmap)
        
        plt.imshow(np.transpose(np.sqrt(u[1:-1,1:-1]**2 + v[1:-1,1:-1]**2)), origin='lower', extent=(0,LX,0,LY), cmap=my_cmap, vmin=0, vmax=1.2)
        plt.colorbar()
        plt.quiver(xx[::4,::4],yy[::4,::4],np.transpose(u[1:-1:4,1:-1:4]),np.transpose(v[1:-1:4,1:-1:4]),scale=40)
        plt.xlim(0,LX)
        plt.ylim(0,LY)
        plt.gca().set_aspect('equal', adjustable='box')
        plt.tight_layout()
 #       plt.savefig('jet4/{:04d}.png'.format((niter+1)//modulo))
        plt.draw()
        
        mpl_pause_background(0.001)